In [2]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn import preprocessing
import urllib

In [3]:
curWeek = 9

In [12]:
P5 = ['ACC','Pac-12','Big Ten','Big 12','SEC','Notre Dame']
G5 = ['Conference USA','Sun Belt','Mid-American','American Athletic','Mountain West','FBS Independents']

def seasonProgression(dfR):
    totalGames = len(dfR)
    print(totalGames)
    curWeek = int(dfR['week']) + 1
    print(curWeek)
    return curWeek/(2*totalGames)

def winningTeam(dfR):
    if dfR['home_points'] > dfR['away_points']:
        return dfR['home_team']
    else:
        return dfR['away_team']
    
def teamWin(dfR, team):
    if dfR.winner == team:
        return True
    else:
        return False
    
def findOpp(dfR):
    if dfR.home == True:
        return dfR.away_team
    else:
        return dfR.home_team
def findOppConf(dfR):
    if dfR.home == True:
        return dfR.away_conference
    else:
        return dfR.home_conference
    
def locMult(dfR):
    if dfR['neutral_site'] == True:
        return 1.05
    elif dfR['home'] == True:
        return 1.1
    else:
        return 1.
    
def pointDiff(dfR,team):
    if dfR.home_team == team:
        return dfR.home_points - dfR.away_points
    else:
        return dfR.away_points - dfR.home_points
    
def confMult(dfR):
    if dfR.opp_conference in P5:
        return 1.
    elif dfR.opp_conference in G5:
        return 0.5
    else:
        return 0.15
    
def winQ(dfR,week): 
    if dfR.opp_conference == None:
        oppWinPct = 0.5
    else:
#         print(dfR.opp_conference)
        oppWinPct = FBSdict[dfR.opp].loc[FBSdict[dfR.opp].week <= week, 'winPct'].tail(1).item()
    
    if dfR.teamWin == True:
        winMult = 1
        oppFactor = oppWinPct
    else:
        winMult = -1
        oppFactor = 1 - oppWinPct
    
    return oppFactor * dfR.seasonProg * dfR.conf_mult * winMult*(dfR.loc_mult + 0.01*dfR.point_diff)

def calcPct(team, week):
    winqweek = 'win_q_'+str(week)
    wins = FBSdict[team].loc[FBSdict[team].week <= week, 'wins'].tail(1).item()
    losses = FBSdict[team].loc[FBSdict[team].week <= week, 'losses'].tail(1).item()
#     print(winqweek,wins,losses,FBSdict[team].loc[FBSdict[team].week <= week, winqweek].mean())
    return 0.9 * FBSdict[team].loc[FBSdict[team].week <= week, winqweek].mean() + 0.01*(wins-losses)

In [4]:
json_url = urllib.request.urlopen('https://api.collegefootballdata.com/games?year=2020&seasonType=regular')
jsonFile = json.loads(json_url.read())
curSch = pd.DataFrame(jsonFile)

In [5]:
# json_url2 = urllib.request.urlopen('https://api.collegefootballdata.com/games?year=2020&seasonType=postseason')
# jsonFile2 = json.loads(json_url2.read())
# postSch = pd.DataFrame(jsonFile2)
# postSch['week'] = postSch['week'] + 15
# curSch = curSch.append(postSch)
# curSch = curSch.dropna(subset=['away_points'])

In [6]:
curSch.loc[curSch.away_team == 'Notre Dame','away_conference'] = 'Notre Dame'
curSch.loc[curSch.home_team == 'Notre Dame','home_conference'] = 'Notre Dame'

In [7]:
curSch.tail(10)

,id,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,venue_id,...,home_points,home_line_scores,home_post_win_prob,away_id,away_team,away_conference,away_points,away_line_scores,away_post_win_prob,excitement_index
599,401251733,2020,15,regular,2020-12-12T05:00:00.000Z,True,False,True,None,3838,...,NaN,None,None,238,Vanderbilt,SEC,NaN,None,None,None
600,401250713,2020,15,regular,2020-12-12T05:00:00.000Z,True,False,True,None,4727,...,NaN,None,None,197,Oklahoma State,Big 12,NaN,None,None,None
601,401249883,2020,15,regular,2020-12-12T05:00:00.000Z,True,False,True,None,3739,...,NaN,None,None,2117,Central Michigan,Mid-American,NaN,None,None,None
602,401249884,2020,15,regular,2020-12-12T05:00:00.000Z,True,False,True,None,3912,...,NaN,None,None,2459,Northern Illinois,Mid-American,NaN,None,None,None
603,401249885,2020,15,regular,2020-12-12T05:00:00.000Z,True,False,True,None,3919,...,NaN,None,None,2711,Western Michigan,Mid-American,NaN,None,None,None
604,401249886,2020,15,regular,2020-12-12T05:00:00.000Z,True,False,True,None,3965,...,NaN,None,None,2006,Akron,Mid-American,NaN,None,None,None
605,401247348,2020,15,regular,2020-12-12T17:00:00.000Z,False,False,True,None,3861,...,NaN,None,None,130,Michigan,Big Ten,NaN,None,None,None
606,401249035,2020,15,regular,2020-12-12T23:00:00.000Z,False,False,True,None,3984,...,NaN,None,None,68,Boise State,Mountain West,NaN,None,None,None
607,401249039,2020,15,regular,2020-12-13T02:30:00.000Z,False,False,True,None,5388,...,NaN,None,None,328,Utah State,Mountain West,NaN,None,None,None
608,401249038,2020,15,regular,2020-12-13T04:00:00.000Z,False,False,True,None,3610,...,NaN,None,None,2439,UNLV,Mountain West,NaN,None,None,None


In [8]:
FBSteams = curSch['home_team'].unique()
FBSteams = np.append(FBSteams,'UMass')
FBSteams.sort()

In [9]:
FBSteams

array(['Air Force', 'Akron', 'Alabama', 'Appalachian State', 'Arizona',
       'Arizona State', 'Arkansas', 'Arkansas State', 'Army', 'Auburn',
       'BYU', 'Ball State', 'Baylor', 'Boise State', 'Boston College',
       'Bowling Green', 'Buffalo', 'California', 'Central Michigan',
       'Charlotte', 'Cincinnati', 'Clemson', 'Coastal Carolina',
       'Colorado', 'Colorado State', 'Duke', 'East Carolina',
       'Eastern Michigan', 'Florida', 'Florida Atlantic',
       'Florida International', 'Florida State', 'Fresno State',
       'Georgia', 'Georgia Southern', 'Georgia State', 'Georgia Tech',
       "Hawai'i", 'Houston', 'Illinois', 'Indiana', 'Iowa', 'Iowa State',
       'Kansas', 'Kansas State', 'Kent State', 'Kentucky', 'LSU',
       'Liberty', 'Louisiana', 'Louisiana Monroe', 'Louisiana Tech',
       'Louisville', 'Marshall', 'Maryland', 'Memphis', 'Miami',
       'Miami (OH)', 'Michigan', 'Michigan State', 'Middle Tennessee',
       'Minnesota', 'Mississippi State', 'Missouri

In [10]:
FBSdict = {}
for team in FBSteams:
    FBSdict[team] = curSch[(curSch['away_team'] == team) | 
                           (curSch['home_team'] == team)]
#     FBSdict[team] = FBSdict[team][np.isfinite(FBSdict[team].away_points)]
    FBSdict[team] = FBSdict[team].reset_index()

In [13]:
seasonProgression(FBSdict['Rutgers'].iloc[0])

25
9


0.18

In [33]:
FBSbar = tqdm(total=len(FBSteams))
for key in FBSdict:
    FBSbar.set_description(key)
    FBSdict[key]['seasonProg'] = FBSdict[key].apply(lambda row: seasonProgression(row),axis = 1)
    FBSdict[key]['winner'] = FBSdict[key].apply(lambda row: winningTeam(row),axis = 1)
    FBSdict[key]['teamWin'] = FBSdict[key].apply(lambda row: teamWin(row,key),axis = 1)
    
    wins = 0
    losses = 0
    winList = []
    lossList = []
    for row in FBSdict[key].itertuples():
    #     print(row.winner)
        if row.teamWin:
            wins += 1
        else:
            losses += 1
        winList.append(wins)
        lossList.append(losses)
        
    FBSdict[key]['wins'] = winList
    FBSdict[key]['losses'] = lossList
    
    hometeam = FBSdict[key]['home_team'] == key
    FBSdict[key]['home'] = hometeam
    
    FBSdict[key]['winPct'] = FBSdict[key]['wins']/(FBSdict[key]['losses'] + FBSdict[key]['wins'])
    FBSdict[key]['opp'] = FBSdict[key].apply(lambda row: findOpp(row), axis = 1)
    FBSdict[key]['opp_conference'] = FBSdict[key].apply(lambda row: findOppConf(row), axis = 1)
    FBSdict[key]['point_diff'] = FBSdict[key].apply(lambda row: pointDiff(row,key), axis = 1)
    FBSdict[key]['loc_mult'] = FBSdict[key].apply(lambda row: locMult(row), axis = 1)
    FBSdict[key]['conf_mult'] = FBSdict[key].apply(lambda row: confMult(row), axis = 1)
    
    FBSbar.update(1)

/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [43]:
[winQ(FBSdict['Ohio State'].iloc[0],i) for i in range(8,15)]

[0.0,
 0.0,
 0.08700000000000001,
 0.06525,
 0.0522,
 0.08700000000000001,
 0.11185714285714286]

In [36]:
FBSbar2 = tqdm(total=len(FBSteams))
for key in FBSdict:
    FBSbar2.set_description(key)
    for w in range(4,curWeek+1):
        try:
            FBSdict[key]['win_q_'+str(w)] = FBSdict[key].apply(lambda row: winQ(row, w),axis=1)
        except ValueError:
            FBSdict[key]['win_q_'+str(w)] = 0.
    FBSbar2.update()

/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [37]:
FBSdict['Ohio State']

,index,id,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,...,opp_conference,point_diff,loc_mult,conf_mult,win_q_4,win_q_5,win_q_6,win_q_7,win_q_8,win_q_9
0,199,401247293,2020,8,regular,2020-10-24T16:00:00.000Z,False,False,True,None,...,Big Ten,35.0,1.1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,280,401247306,2020,9,regular,2020-10-31T23:30:00.000Z,False,False,True,None,...,Big Ten,13.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,302,401247310,2020,10,regular,2020-11-07T05:00:00.000Z,True,False,True,None,...,Big Ten,NaN,1.1,1.0,0.0,0.0,0.0,0.0,NaN,NaN
3,378,401247318,2020,11,regular,2020-11-14T05:00:00.000Z,True,False,True,None,...,Big Ten,NaN,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN
4,448,401247326,2020,12,regular,2020-11-21T05:00:00.000Z,True,False,True,None,...,Big Ten,NaN,1.1,1.0,0.0,0.0,0.0,0.0,NaN,NaN
5,510,401247331,2020,13,regular,2020-11-28T05:00:00.000Z,True,False,True,None,...,Big Ten,NaN,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN
6,544,401247340,2020,14,regular,2020-12-05T05:00:00.000Z,True,False,True,None,...,Big Ten,NaN,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN
7,605,401247348,2020,15,regular,2020-12-12T17:00:00.000Z,False,False,True,None,...,Big Ten,NaN,1.1,1.0,0.0,0.0,0.0,0.0,NaN,NaN


In [50]:
calcPct('Alabama',9)

0.14146400000000003

In [52]:
FBSbar3 = tqdm(total=len(FBSteams))
FBSpct = {}
for key in FBSdict:
    FBSbar3.set_description(key)
    pctList = []
    for w in range(4,curWeek):
        try:
            pctList.append(calcPct(key,w))
        except ValueError:
            pctList.append(0.)
    FBSpct[key] = pctList
    FBSbar3.update()

/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [55]:
FBSpct

{'Air Force': [0.0, 0.01, 0.01, 0.01, 0.019305],
 'Akron': [0.0, 0.0, 0.0, 0.0, 0.0],
 'Alabama': [0.01,
  0.057260000000000005,
  0.09111999999999999,
  0.117985,
  0.1311296],
 'Appalachian State': [0.0169225,
  0.012385,
  0.0049724999999999995,
  0.0047625,
  0.025124821428571427],
 'Arizona': [0.0, 0.0, 0.0, 0.0, 0.0],
 'Arizona State': [0.0, 0.0, 0.0, 0.0, 0.0],
 'Arkansas': [-0.01,
  0.028890000000000002,
  -0.010880000000000003,
  -0.00190125,
  -0.0010102500000000003],
 'Arkansas State': [-0.009875,
  -0.020999999999999998,
  -0.008937875,
  0.007795700000000001,
  -0.006925249999999997],
 'Army': [0.0,
  0.0128755,
  0.025568300000000002,
  0.04120825,
  0.05437437755102041],
 'Auburn': [0.01, 0.0, 0.03828, 0.009922500000000004, 0.019954000000000006],
 'BYU': [0.03, 0.04, 0.05, 0.06, 0.10631571428571429],
 'Ball State': [0.0, 0.0, 0.0, 0.0, 0.0],
 'Baylor': [-0.01, -0.03692, -0.03692, -0.04269, -0.07275999999999999],
 'Boise State': [0.0, 0.0, 0.0, 0.0, 0.01],
 'Boston Colleg

In [57]:
PCTdf = pd.DataFrame.from_dict(FBSpct,orient='index',columns=range(4,curWeek))

In [59]:
PCTdf.to_csv('2020/PCT.csv')

In [63]:
PCTnormdf = pd.DataFrame(index = FBSteams,columns=range(4,curWeek))

for w in range(4,curWeek):
    x = PCTdf[w].values.reshape(-1,1)
    min_max_scaler = preprocessing.MinMaxScaler()
    xScaled = min_max_scaler.fit_transform(x)
    PCTnormdf[w] = xScaled.reshape(127)

PCTnormdf.to_csv('2020/PCT_norm.csv')

In [71]:
PCTnormdf.sort_values(8,ascending=False).loc['Oregon']

4    0.497082
5    0.448728
6    0.415021
7    0.354448
8    0.357534
Name: Oregon, dtype: float64

In [79]:
PCTdf.rank(method='first',ascending=False)

,4,5,6,7,8
Alabama,23.0,8.0,2.0,1.0,1.0
Clemson,14.0,4.0,3.0,2.0,2.0
Notre Dame,6.0,11.0,6.0,3.0,3.0
BYU,9.0,13.0,12.0,12.0,4.0
Oklahoma State,3.0,3.0,8.0,5.0,5.0
...,...,...,...,...,...
Rice,78.0,NaN,NaN,NaN,123.0
Baylor,101.0,119.0,113.0,116.0,124.0
Kansas,122.0,120.0,121.0,125.0,125.0
Colorado State,49.0,51.0,51.0,53.0,NaN


In [80]:
Ranks = PCTdf.rank(method='first',ascending=False).fillna(999).astype('int64')
Ranks.to_csv('2020/Ranks.csv')

In [81]:
for w in range(4,curWeek):
    Wdf = pd.concat([PCTnormdf[w],Ranks[w]],axis=1)
    Wdf.columns = ['PCT','Rank']
    Wdf = Wdf.sort_values('Rank')
    Wdf.to_csv('2020/W%s.csv'%w)

In [83]:
Ranks.sort_values(8).head(30)

,4,5,6,7,8
Alabama,23,8,2,1,1
Clemson,14,4,3,2,2
Notre Dame,6,11,6,3,3
BYU,9,13,12,12,4
Oklahoma State,3,3,8,5,5
Cincinnati,16,17,10,10,6
Coastal Carolina,15,14,19,9,7
Liberty,17,18,22,11,8
Miami,7,2,21,7,9
Marshall,18,34,33,14,10
